[](http://)## Fairfield County Random Forest Traffic Analysis 
by [David Roberts](https://www.kaggle.com/davidroberts13) Sep 2020

*In this notebook, we will explore, manipulate, and analyze highway crashes in CT's Fairfield County. More specifically, their influence on traffic. Through the utilization of a Random forest, our goal is to accurately predict the severity or level of impact on traffic an accident has.*

![s](https://mocah.org/uploads/posts/5322598-forest-traffic-road-tree-windy-road-night-winter-snow-nature-light-trail-long-exposure-dark-landscape-pine-fur-wood-fujifilm-night-light-traffic-road-car-light-free-pictures.jpg)


### Table of Contents

* [1.Introduction](#chapter1)
   * [1.1 Load and Check Data](#section_1_1)
   * [1.2 Scope](#section_1_1)
   * [1.3 Data Type Manipulation](#section_1_3)
* [2. Feature Engineering](#chapter2)
    * [2.1 Missing Values and Imputation](#section_2_1)
    * [2.2 Handling Outliers (Weather/City)](#section_2_2)
    * [2.3 Extracting Date](#section_2_3)
    * [2.4 Binning](#section_2_4)
    * [2.5 One-Hot Encoding](#section_2_5)
* [3. Visulaization](#chapter4)
    * [3.1 Geospacial (Folium)](#section_3_1)
    * [3.2 Word Clouds](#section_3_2)
* [4. Machine Learning](#chapter5)
    * [4.1 Model Selection](#section_4_1)
    * [4.2 Training](#section_4_2)
    * [4.3 Model Evaluation](#section_4_3)
* [5. Conclusion](#chapter6)

   

## 1.Introduction <a class="anchor" id="chapter1"></a>

#### *Business Problem:*
   Automotive collisions have become a ubiquitous aspect of modern life. These accidents claim the lives and livelihoods of millions of people a year all over the world.  A study by the Department of Transportation's, [National Highway Traffic Saftey Administration](https://crashstats.nhtsa.dot.gov/Api/Public/ViewPublication/812013) (NHTSA), placed a price tag of over a quarter-trillion dollars in 2010.  I have seen several projects diving into this problem from the perspective of loss of life property or employment, all of which are noble in nature. I want to explore something else, something more trivial, and infuriating TRAFFIC! That's right, and I will be analyzing and building a machine learning model to predict an accident's impact on traffic. Through analysis of observations like weather, time of year, location, and many more, we might be able to alleviate the modern leech on our precious free time.
   
   By better understanding the critical factors influencing a collision effect on traffic flow, we can reduce the number of total crashes and limit the time they steal from us. If done correctly, this work could help add precious time to our days.  To defeat them, we first understand them as Sun Tzu said, "know your opponent, and you will never lose'. We must learn what makes traffic tick, and we will do it through careful feature engineering and model choice. If we can establish a pattern,  we would deploy available resources to combat these conditions that worsen traffic. 

This analysis will help inform the driver of critical areas better and plan their route accordingly and decision-makers on the local level and above on resource allocation. 
   
   *TLDR* Car crashes have many issues associated with them that are horrible and life-changing. Traffic, however, affects us the most regularly. This analysis will aim to understand better the conditions that make a crash cause a higher traffic load.

#### *Data:*
This US-Accident Dataset is a countrywide dataset covering 49 US states. It has 3.5 million separate traffic accidents taking place from **February 2016** to **December 2019**. 
Based on definition of our problem, factors that will influence our decission are:
* Information describing the collision
* Various measurements and observations of the weather
* Types of roadway or infrastructure involved 
We choose to use the data's entirety in our predictive efforts to better our target audience of governmental decision-makers. Remember, severity is only relational to the impact an accident has on traffic flow and will be our target variable in our later analysis.

#### *Acknowledgments:*
Moosavi, Sobhan, Mohammad Hossein Samavatian, Srinivasan Parthasarathy, and Rajiv Ramnath. "A Countrywide Traffic Accident Dataset.", 2019. Moosavi, Sobhan, Mohammad Hossein Samavatian, Srinivasan Parthasarathy, Radu Teodorescu, and Rajiv Ramnath. "Accident Risk Prediction based on Heterogeneous Sparse Data: New Dataset and Insights." In Proceedings of the 27th ACM SIGSPATIAL International Conference on Advances in Geographic Information Systems, ACM, 2019.

### 1.1 Load and Check Data <a class="anchor" id="section_1_1"></a>

In [ ]:
#required librarys
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib as mpl
%matplotlib inline

from sklearn.preprocessing import LabelEncoder #For Label Encoding 
#path to file
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

# Import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
pd.set_option('display.max_columns', None) #allows us to see all the columns avoiding the dreded '...'

#Word Cloud 
from PIL import Image
from wordcloud import WordCloud, STOPWORDS

#Reading Data
df=pd.read_csv('/kaggle/input/us-accidents/US_Accidents_June20.csv')

df.head(3)

In [ ]:
print('The shape of the raw data from the dataset is:',df.shape,',and that is just not gonna happen!')

### 1.2 Scope <a class="anchor" id="section_1_2"></a>
Now, as you could see, this data set is far too large for my old laptop to handle without catching my desk on fire, so we will need to widdle it down to a more appropriate level. We will be focusing our efforts on just Fairfield county, the most South Western county in Connecticut with several larger(ish) Cities and many commuters. 

In [ ]:
#First we will subset our data down to just the state level
df_CT=df.loc[df['State']=='CT'].copy()
#County level 
df_FF=df_CT.loc[df_CT['County']=='Fairfield'].copy()
print('The new shape after defining the scope:',df_FF.shape,',that is something i can work with')
pd.set_option('display.max_columns', None)
df_FF.head(1)

In [ ]:
print(df_FF.head())
print(df_FF.describe())
print(df_FF.columns)
print(df_FF.info())

In [ ]:
print('if your interested in seeing some descriptive statistics on the overall data set click the following code button bellow')

### 1.3 Boolean Manipulation <a class="anchor" id="section_1_3"></a>

*As we can see in the head()'s that we've shown so far, we have many Boolean operations, but how useful are they?*

In [ ]:
print(df_FF.shape)
print(df_FF['Amenity'].value_counts())#Only 20 True Observations 
print(df_FF['Bump'].value_counts())#No True Observations 
print(df_FF['Crossing'].value_counts())#86 True Observations
print(df_FF['Give_Way'].value_counts())#No True Observations 
print(df_FF['Junction'].value_counts())#807 True Observations
print(df_FF['No_Exit'].value_counts())#Only 1 True Observations
print(df_FF['Railway'].value_counts())#Only 5 True Observations
print(df_FF['Roundabout'].value_counts())#No True Observations 
print(df_FF['Station'].value_counts())#Only 3 True Observations
print(df_FF['Stop'].value_counts())#Only 31 True Observations
print(df_FF['Traffic_Calming'].value_counts())#No True Observations 
print(df_FF['Traffic_Signal'].value_counts())#181 True Observations
print(df_FF['Turning_Loop'].value_counts())#No True Observations 

As we can see above 10 of the 13 columns have 1% or less true values. this is not substantial enough to keep them in the analysis. The 3 that demonstrate a significant amount will stay for further manipulation. 

In [ ]:
#The 10 columns that are almost entirely false and thus not useful
df_FF.drop(columns=['Amenity'],axis=1,inplace=True)
df_FF.drop(columns=['Bump'],axis=1,inplace=True)
df_FF.drop(columns=['Stop'],axis=1,inplace=True)
df_FF.drop(columns=['Give_Way'],axis=1,inplace=True)
df_FF.drop(columns=['No_Exit'],axis=1,inplace=True)
df_FF.drop(columns=['Railway'],axis=1,inplace=True)
df_FF.drop(columns=['Roundabout'],axis=1,inplace=True)
df_FF.drop(columns=['Station'],axis=1,inplace=True)
df_FF.drop(columns=['Traffic_Calming'],axis=1,inplace=True)
df_FF.drop(columns=['Turning_Loop'],axis=1,inplace=True)

#Just a couple of extra columns that do not give us enough information on their own to warrant staying.
df_FF.drop(columns=['Nautical_Twilight'],axis=1,inplace=True) #Closely associated with Sunset, so they will not add to our analysis 
df_FF.drop(columns=['Astronomical_Twilight'],axis=1,inplace=True)
df_FF.drop(columns=['Civil_Twilight'],axis=1,inplace=True)

#Now to convert the remaining Boolean objects into intiger objects. 
df_FF['Junction']=df_FF['Junction'].astype(int)
df_FF['Crossing']=df_FF['Crossing'].astype(int)
df_FF['Traffic_Signal']=df_FF['Traffic_Signal'].astype(int)


In [ ]:
df_FF.head(1)

We can see that we are making significant progress. However, there are still many columns that we can see will not help the analysis we are doing, so we will drop them.

In [ ]:
df_FF.drop(columns=['ID'],axis=1,inplace=True) #Superfluous 
df_FF.drop(columns=['Country'],axis=1,inplace=True) #This is a uniform column displaying just the 'US'
df_FF.drop(columns=['State'],axis=1,inplace=True) #This is a uniform column displaying just 'CT' 
df_FF.drop(columns=['County'],axis=1,inplace=True)#This is a uniform column displaying just 'Fairfield'
df_FF.drop(columns=['Timezone'],axis=1,inplace=True) #This is a uniform column displaying just 'US/Eastern'
df_FF.drop(columns=['Airport_Code'],axis=1,inplace=True)#Who cares about airport code, am I right?
df_FF.drop(columns=['Weather_Timestamp'],axis=1,inplace=True)#This relates to what time the weather data was recorded

In [ ]:
df_FF.head(1)

Now, this is starting to look good! Don't you think?

## 2.Feature Engineering <a class="anchor" id="chapter2"></a>

Generally speaking, this is the process of extracting features from raw data using domain knowledge. We will do so much, but first, we should see how tidy our data look for that I like to use a nifty little visualization. It will show us a big blue box with yellow lines representing a missing value in that category. Our goal is to remove all these yellow lines. 

In [ ]:
font = {'family': 'serif',
        'color':  'darkred',
        'weight': 'normal',
        'size': 36,
        }
plt.figure(figsize=(30,12))
sns.heatmap(df_FF.isnull(),yticklabels=False,cbar=False,cmap='plasma')
plt.title('Missing Values Visualized', fontdict=font)
plt.show()

Okay, not too bad! 13 columns are affected by missing values. We will outright remove columns like he 'End_Lat/Lng' and 'Number' column as they are not useful. Then some will need imputation work like 'Wind_Speed(mph),' 'Wind_Chill(F),' and others. Some of these like 'TMC,' 'Weather_Conditions,' will require a closer examination.

![](http://)

In [ ]:
#Far to many missing values to keep these columns and they dont add much to our analysis so the decision is easy
df_FF.drop(columns=['End_Lat','End_Lng', 'Number'],axis=1,inplace=True) 
df_FF.head(1)

### 2.1 Missing Values and Imputation <a class="anchor" id="section_2_1"></a>
We are not sure why these pieces of data are missing, but we know they must go. Imputation is a critical skill in preprocessing data as Machine Learning Algorithms are allergic to them, so they must go. Statistical imputation's most frequent strategy is replacing missing values with the mean value of the column. This technique gives a rough fix for the missing value but only works if there aren't too many missing data pieces.

In [ ]:
df_FF['Wind_Chill(F)'].fillna(df_FF['Wind_Chill(F)'].mean(),inplace=True)
df_FF['Wind_Speed(mph)'].fillna(df_FF['Wind_Speed(mph)'].mean(),inplace=True)
df_FF['Visibility(mi)'].fillna(df_FF['Visibility(mi)'].mean(),inplace=True)
df_FF['Precipitation(in)'].fillna(df_FF['Precipitation(in)'].mean(),inplace=True) 
df_FF['Temperature(F)'].fillna(df_FF['Temperature(F)'].mean(),inplace=True)
df_FF['Humidity(%)'].fillna(df_FF['Humidity(%)'].mean(),inplace=True)
df_FF['Pressure(in)'].fillna(df_FF['Pressure(in)'].mean(),inplace=True)

Ps. Precipitation has a rather high amount of missing vlaues but our area has rather low precipitation numbers and if precipitation was a factor in the crash it would likey be mentioned in the report. Odds are if it was left out it was not a factor

### 2.2 Extracting Date  <a class="anchor" id="section_2_2"></a>
DateTime data comes in many forms. It's paramount we can grab these values no matter how they are written and massage them into something we can use. I find Pandas to do this exceptionally well.

In [ ]:
#Converting the giving time data into workable Date-time objects
df_FF['Start_Time']=pd.to_datetime(df_FF['Start_Time'],infer_datetime_format=True)

#Breaking the start end time into usable independant features 
df_FF['Year'] = df_FF['Start_Time'].dt.year
df_FF['Month'] = df_FF['Start_Time'].dt.month
df_FF['Day'] = df_FF['Start_Time'].dt.day
df_FF['Time_S'] = df_FF['Start_Time'].dt.hour
df_FF['Weekday']=df_FF['Start_Time'].dt.weekday
df_FF.drop(columns=['Start_Time','End_Time'],axis=1,inplace=True) # supurfluous now


### 2.3 One-Hot Encoding <a class="anchor" id="section_2_3"></a>

Categorical data like color 'Red,' 'Blue,' 'Green,' or in our case, we are looking at 'Weather_Conditions' and 'City' is standard. This data has a rich amount of information in it that we can understand as smart monkeys. On the other hand, our computers can't make heads or tails of it, at least at my current level. To enlist the help of our computers, we must use these categorical columns into numeric ones. So we must change them into a langue our computers can understand like NUMBERS! We have a couple of options [here](https://machinelearningmastery.com/how-to-prepare-categorical-data-for-deep-learning-in-python/) l like Integer Encoding, mapping each label to an integer, One Hot encoding. Each label gets mapped to a binary vector. Learned Embedding, where a distributed representation of the categories are discovered and extracted. For our purpose, One hot is the best bet as its most effective when there is no relationship between the categorical variables. This type of conversion makes the data a tad unwieldy, so we'll save the conversion for later, but we will be finalizing its prep in this section.

We will handle 'Weather_Condition' first. But I seem to remember that we need to fix a handful of missing values before moving on to One-Hot.

In [ ]:
df_FF[df_FF['Weather_Condition'].isnull()]

Ahhh, yes, we might find some. These are simple errors and seem to all be from the same time and day. Which makes it easy we will be able to replace them all with the same value. But we must make sure we get the language right.

In [ ]:
df_FF['Weather_Condition'].fillna(method='ffill',inplace=True) 
df_FF['Wind_Direction'].fillna(method='ffill',inplace=True) #We also have one missing value here we need to fix real quick

### 2.4 Binning <a class="anchor" id="section_2_4"></a>

Binning is another classic weapon in the data scientists' arsenal. This technique is best suited to transform categorical values into subset groups extracting information. For example, grouping credit scores into excellent, fair, and bad allows for new analysis on these bins or buckets. It can also be called bucketing.

In [ ]:
#Define bins for time (24 hour time) 0 to 6, 6-12, 12-18,18-24
timeBins=[-1,6,12,18,24]
tBin_names=['Early Morning','Morning','Afternoon','Evening']
df_FF['TimeofDay']=pd.cut(df_FF['Time_S'],timeBins,labels=tBin_names)
df_FF.head(5)

In [ ]:
#Define bins for Season (Months) 0 to 2, 2-5, 5-8,8-11,11-12
seasonBins=[-1,2,5,8,11,12]
sBin_names=['Winter','Spring','Summer','Autumn','Winter']
df_FF['Season']=pd.cut(df_FF['Month'],seasonBins,labels=sBin_names,ordered=False)
df_FF.tail(5)

In [ ]:
#Define bins for Day_type -1(Mon) to 4(Fri),4(Fri)-6(Sun)
seasonBins=[-1,4,6]
sBin_names=['Weekday','Weekend']
df_FF['Day_Type']=pd.cut(df_FF['Weekday'],seasonBins,labels=sBin_names,ordered=False)
df_FF.tail(5)

Now we can quickly see what section of the day the accident happened durring and what season of the year. lets quickly examine these new categorys

In [ ]:
fig, ax =plt.subplots(2,3,figsize=(25,10))
T=sns.countplot(x='TimeofDay',hue='Severity',data=df_FF,ax=ax[0][0],palette='rocket_r')
T.legend(loc='upper right',bbox_to_anchor=(.6, 0.5, 0.5, 0.5))    
t=sns.countplot(x='Time_S',hue='Severity',data=df_FF,ax=ax[1][0],palette='rocket_r')
t.legend(loc='upper right',bbox_to_anchor=(.6, 0.5, 0.5, 0.5))   
S=sns.countplot(x='Season',hue='Severity',data=df_FF,ax=ax[0][1],palette='mako_r')
S.legend(loc='upper right',bbox_to_anchor=(.6, 0.5, 0.5, 0.5))
s=sns.countplot(x='Month',hue='Severity',data=df_FF,ax=ax[1][1],palette='mako_r')
s.legend(loc='upper right',bbox_to_anchor=(.6, 0.5, 0.5, 0.5))
W=sns.countplot(x='Day_Type',hue='Severity',data=df_FF,ax=ax[0][2],palette='rocket_r')
W.legend(loc='upper right',bbox_to_anchor=(.6, 0.5, 0.5, 0.5))
w=sns.countplot(x='Weekday',hue='Severity',data=df_FF,ax=ax[1][2],palette='rocket_r')
w.legend(loc='upper right',bbox_to_anchor=(.6, 0.5, 0.5, 0.5))


fig.show()


We can see both the binned or bucketed data and its raw counterpart. On the left, we have two Seaborn Countplots that show us the relation of commuting and car accident frequency and severity. We can see that the afternoon seems to be a more difficult time of day. Understandably we can expect drivers to be more tired and thus more impaired after a full day of work. One interesting takeaway from the plots on the right is that Autumn seems to be a more dangerous time to be on the road. We found this interesting because our gut would have said that New England winters would have caused a more considerable uptick in accidents than the fall weather. Possibly due to holidays involving travel and the influx of 'Leafers' people who drive through CT to get to the more northern states to see the leaf change color. 

P.S. I mean no office to all the leaf people out there. 

Now we have one last column to handle before we can have some sweet tidy data before moving on to our final feature, and that is 'TMC' or [Traffic Message Channel](https://en.wikipedia.org/wiki/Traffic_message_channel#:~:text=Traffic%20Message%20Channel%20(TMC)%20is,via%20conventional%20FM%20radio%20broadcasts.)
 TMC is a way to communicate traffic information with drivers and observers via FM radio waves originally from Europe. It consists of [thousandas](https://wiki.openstreetmap.org/wiki/TMC/Event_Code_List) of individual packets of data from code: 135 traffic easing to code 1595: boat race. This vast range means imputing a code into the missing values here will be impossible. So we have two choices remove the column, or remove the rows with the missing values. Lets soo how many missing values we have before making this choice.

In [ ]:
print('we have',df_FF['TMC'].isna().sum(), 'missing values in the TMC column')
print('out of a total',df_FF.shape)

That is just too much data to sacrifice to keep the column. We will remove the TMC observation; then, we should have clean and tidy data. We will rerun the missing value visualization to double-check.

In [ ]:
df_FF.drop(columns='TMC',inplace=True)

In [ ]:
font = {'family': 'serif',
        'color':  'darkred',
        'weight': 'normal',
        'size': 36,
        }
plt.figure(figsize=(30,12))
sns.heatmap(df_FF.isnull(),yticklabels=False,cbar=False,cmap='plasma')
plt.title('Missing Values Visualized', fontdict=font)
plt.show()

One Big Ocean perfect! We are ready to move on to some Dummy variables. Oh yeah, and we should make our Sunrise and Side column into integers.

In [ ]:
df_FF['Sunrise_Sunset'].replace('Night',0,inplace=True)
df_FF['Sunrise_Sunset'].replace('Day',1,inplace=True)

df_FF['Side'].replace('L',0,inplace=True)
df_FF['Side'].replace('R',1,inplace=True)

Now we are ready!

## 3.Visualizations <a class="anchor" id="chapter3"></a>



### 3.1 Folium Maps <a class="anchor" id="section_3_1"></a>
The cartography class is in session. Folium is a fantastic hybrid of Python data wrangling and Leaflet; its strengths lay in the mapping department. I highly recommend [reading](https://python-visualization.github.io/folium/) up to about this beautiful little library if you haven't yet. I tried to run just our subsetted dataset through my folium maps and nearly crashed my computer. So we will need to randomly sample our df_FF to make it a more manageable size. Let's start with half of what it currently is.

In [ ]:
#Randomly subsetting the data to make it more manageable 
df_FF_half=df_FF.sample(frac=0.5, replace=True, random_state=101)
df_FF_half.head(1)

In [ ]:
import folium
#Creating map of our locaiton of choice Fairfield County
FF_map = folium.Map(location=[41.40,-73.263],tiles = 'Stamen Terrain', zoom_start=10)
FF_map

from folium import plugins
from folium.plugins import HeatMap

#Making sure our data is in the correct type
df_FF_half['Start_Lat']=df_FF_half['Start_Lat'].astype(float)
df_FF_half['Start_Lng']=df_FF_half['Start_Lng'].astype(float)

#Subsetting data for visualization
df_FFHeat=df_FF_half[['Start_Lat','Start_Lng']]
df_FFHeat=df_FFHeat.dropna(axis=0,subset=['Start_Lat','Start_Lng'])

#Creating and Attaching heatmap to our map
FFHeat_data=[[row['Start_Lat'],row['Start_Lng']] for index, row in df_FFHeat.iterrows()]
HeatMap(FFHeat_data,blur=10,radius=15,gradient={0.4: 'green', 0.65: 'yellow', 1: 'red'}).add_to(FF_map)

#show
FF_map


*Don't be afraid to get in there and scroll around! Do you see any patterns?*

This next graph is so CPU intensive I'm going to reduce the data to 1/8 to alleviate machine load. By the way, shout out to [Dave Fisher-Hickey](https://www.kaggle.com/daveianhickey/how-to-folium-for-maps-heatmaps-time-analysis) for his amazing folium explainer it helped a lot with this map.

In [ ]:
#Reducing data to 1/8 its sice
df_FF_e=df_FF.sample(frac=0.125, replace=True, random_state=101)
df_FF_e.head(1)

#Same Map generation as before
FF_map2 = folium.Map(location=[41.40,-73.263],tiles = 'Stamen Terrain', zoom_start=10)

#Marker Creation
Marker_Map = folium.Map(location=[41.40,-73.263],tiles = 'Stamen Terrain', zoom_start=10)
for i in range(0,len(df_FF_e)):
    folium.Marker([df_FF_e['Start_Lat'].iloc[i],df_FF_e['Start_Lng'].iloc[i]]).add_to(Marker_Map) 
Marker_Map


#Adding labels and colors to our markers 
for i in range(0,len(df_FF_e)):
    Severity = df_FF_e['Severity'].iloc[i]
    if Severity == 1:
        color = 'green'
        popup = folium.Popup('Severity 1', parse_html=True) 
    elif Severity == 2:
        color = 'orange'
        popup = folium.Popup('Severity 2', parse_html=True) 

    elif Severity == 3:
        color = 'red'
        popup = folium.Popup('Severity 3', parse_html=True) 

    else:
        color = 'purple'
        popup = folium.Popup('Severity 4', parse_html=True) 

    
    #Adding our code to the map
    folium.Marker([df_FF_e['Start_Lat'].iloc[i],df_FF_e['Start_Lng'].iloc[i]],popup=popup,icon=folium.Icon(color=color, icon='info-sign')).add_to(Marker_Map)
#Display
Marker_Map

Unsurprisingly as significant highways go through cities, we see a spike in traffic inducing accidents. Can you see anything else?

### 3.2 Word Clouds <a class="anchor" id="section_3_2"></a>
Word Clouds are so much more than a fun way to visualize data. They can help quickly and digestible glean frequency insights for a portion of text.  For this data set, we have a column named description, which lends itself Word-Cloud visualization. The more text data you have, the better a word cloud will perform, so let's check how many of each severity we have. 


Ps. Shouts out to [Aashita Kesarwani](https://www.kaggle.com/aashita) for the assistance getting these to look juuust right.


In [ ]:
print(df_FF['Severity'].value_counts())
print('\n')
print('It looks like we should have enough text data for a Word Cloud of Severity levels 2, 3, and 4.')

In [ ]:
df_FF_S4=df_FF[df_FF['Severity']==4] #Making subsets for us to use in our wordcloud generation 
df_FF_S3=df_FF[df_FF['Severity']==3]
df_FF_S2=df_FF[df_FF['Severity']==2]


#Custome Color Map for Severity 2
cmap_O = mpl.cm.Oranges(np.linspace(0,1,20))
cmap_O = mpl.colors.ListedColormap(cmap_O[10:,:-1])

#Custome Color Map for Severity 3
cmap_R = mpl.cm.Reds(np.linspace(0,1,20))
cmap_R = mpl.colors.ListedColormap(cmap_R[10:,:-1])

#Custome Color Map for Severity 4
cmap_H = mpl.cm.hot_r(np.linspace(0,1,20))
cmap_H = mpl.colors.ListedColormap(cmap_H[10:,:-1])

In [ ]:
#Font parameters for our Title
font = {'family': 'serif',
        'color':  'darkred',
        'weight': 'normal',
        'size': 36,
        }

#Creating variable with data for this Word Cloud
textS2 = ' '.join(df_FF_S2['Description'].tolist())

#Creating Mask for Word Cloud
d = '../input/car-silhouette/'
car_mask = np.array(Image.open(d + 'Car_mask3.jpg'))
stop_words=set(STOPWORDS)

#Word Cloud Creation 
Car_wc=WordCloud(width=400,height=200,mask=car_mask,random_state=101, max_font_size=450,
                 min_font_size=1,stopwords=stop_words,background_color="white",
                 scale=3,max_words=400,collocations=True,colormap=cmap_O)

#Generate Word Cloud
Car_wc.generate(str(textS2))



#show
fig=plt.figure(figsize=(30,10))
plt.ylim(-250,2700)
#plt.xlim(0,2650)
plt.gca().invert_yaxis()
#plt.gca().invert_xaxis()
plt.axis("off")
plt.title('Accidents: Severity 2',fontdict=font)
plt.imshow(Car_wc,interpolation='bilinear')
plt.show()


In [ ]:
#Creating variable with data for this Word Cloud
textS3 = ' '.join(df_FF_S3['Description'].tolist())

#Creating Mask for Word Cloud
d = '../input/car-silhouette/'
car_mask = np.array(Image.open(d + 'Car_mask2.jpg'))
stop_words=set(STOPWORDS)

#Word Cloud Creation 
Car_wc=WordCloud(width=400,height=200,mask=car_mask,random_state=101, max_font_size=450,
                 min_font_size=1.5,stopwords=stop_words,background_color="white",
                 scale=3,max_words=400,collocations=True,colormap=cmap_R)

#Generate Word Cloud
Car_wc.generate(str(textS3))



#Show
fig=plt.figure(figsize=(30,10))
#plt.ylim(-300,2600)
#plt.xlim(0,7500)
#plt.gca().invert_xaxis()
#plt.gca().invert_yaxis()
plt.axis("off")
plt.title('Accidents: Severity 3',fontdict=font)
plt.imshow(Car_wc,interpolation='bilinear')
plt.show()

In [ ]:
#Creating variable with data for this Word Cloud
textS4 = ' '.join(df_FF_S4['Description'].tolist())

#Creating Mask for Word Cloud
d = '../input/car-silhouette/'
car_mask = np.array(Image.open(d + 'Car_mask1.jpg'))
stop_words=set(STOPWORDS)

#Word Cloud Creation 
Car_wc=WordCloud(width=400,height=200,mask=car_mask,random_state=101, max_font_size=450,
                 min_font_size=1.5,stopwords=stop_words,background_color="white",
                 scale=3,max_words=400,collocations=True,colormap=cmap_H)

#Generate Word Cloud
Car_wc.generate(str(textS4))


#Show
fig=plt.figure(figsize=(30,10))
plt.ylim(500,1300)
plt.gca().invert_yaxis()
plt.axis("off")
plt.title('Accidents: Severity 4',fontdict=font)
plt.imshow(Car_wc,interpolation='bilinear')
plt.show()

**Severity 2:** We see words relating to partially blocked sections or road mostly on the right lane. The exit is also a significant contributor to this cloud. It seems as though these accidents happen when people are trying to get off the highway. 

**Severity 3:** The significant distinction we see it phrasing changed from blocked or obscured to closed. Yet stopped is still a factor. We also see lane closed coming up with some regularity.

**Severity 4:** The most critical term that jumps out to us as a differentiating factor is 'Road Closed' no longer are talking about lanes closure of blocks. Severity 4 means things are coming to a standstill. 

## 4.Machine Learning <a class="anchor" id="chapter4"></a>
Now for the grand finale, the Machine Learning the reason you are all here. Id likes to start and say thank you for reading this far you are part of a select few: the chosen the elite group of data connoisseur. I salute to you!

### 4.1 Model Selection <a class="anchor" id="section_4_1"></a>
Our choice of random forest is anything but random. I find this model so good I'm actively working to make sure I stay more agnostic in my applications. Still, this here isn't one of those situations. Random forests default hyperparameters seem to hit it out of the par more often than they miss. In 1995 the brilliant [Tin Kam Ho](https://en.wikipedia.org/wiki/Tin_Kam_Ho) created the first random forest making this method the same age as me, another star on the wall of the random forest. There are some definite advantages to the RFC. The main reason the Random Forest is that it's favorable for data sets with high dimensionality like this one! 

### 4.2 Training <a class="anchor" id="section_4_2"></a>
First, we need to break our data set into two separate pieces a training and testing set. We will use this training data to hone in our Random Forest before seeing how we did with the test data.

In [ ]:
df_FF.head(1)

In [ ]:
df_FF.columns

Now we must first select the features we would like to use in our Random Forest. Then we need to make sure our forest knows what to do with said features. We put much work in at the beginning to make as many of the columns as we could into good old numbers, but some are still in categorical form. Remember when we talked about One-Hot encoding, but we would save it for later. Well, it's later, and now we must purge our data of these complicated categorical columns for clean 1's and 0's. We will be using a Pandas method called get_dummies to do our heavy lifting here.

In [ ]:
#What we are trying to predict
target='Severity'

featuers_removed=['Description', 'Street','Zipcode','Source','Year', 'Month', 'Day', 
          'Time_S', 'Weekday']

#What we are using to predict our target
features=['Severity', 'Start_Lat', 'Start_Lng', 'Distance(mi)',
          'Side', 'City','Temperature(F)','Wind_Chill(F)', 'Humidity(%)',
          'Pressure(in)', 'Visibility(mi)','Wind_Direction', 'Wind_Speed(mph)',
          'Precipitation(in)','Weather_Condition', 'Junction', 'Crossing', 
          'Traffic_Signal','Sunrise_Sunset','TimeofDay', 'Season', 'Day_Type'] 

#One-Hot Encoding 
df_FF_Dummy=pd.get_dummies(df_FF[features],drop_first=True)
print(df_FF_Dummy.info())
df_FF_ML = df_FF_Dummy.reset_index()
df_FF_ML=df_FF_ML.drop('index',axis=1)
df_FF_ML.fillna(0)

#Train Test Split is a great function to break our data down. i made test 30% of total
y=df_FF_ML[target]
X=df_FF_ML.drop(target, axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [ ]:
#Running Model object
clf=RandomForestClassifier(n_estimators=250)

#Train Model with data
clf.fit(X_train,y_train)

#Run Model to predict accident Severity
predictions=clf.predict(X_test)

# Get the accuracy score
acc=accuracy_score(y_test, predictions)


# Model Accuracy, how often is the classifier correct?
print("[Randon forest algorithm] accuracy_score:",accuracy_score(y_test, predictions))
print('\n')
print('Confusion Matrix of results')
print(confusion_matrix(y_test,predictions))
print('\n')
print('Classificaiton Report of results')
print(classification_report(y_test,predictions))


### 4.3 Model Evaluation <a class="anchor" id="section_4_3"></a>

In [ ]:
feature_imp = pd.Series(clf.feature_importances_,index=X.columns).sort_values(ascending=False)
# Creating a bar plot, displaying only the top k features
k=10
sns.barplot(x=feature_imp[:10], y=feature_imp.index[:k])
# Add labels to your graph
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.title("Visualizing Important Features")
plt.legend()
plt.show()

## 5.Conclusion <a class="anchor" id="chapter5"></a>

When it comes to limiting traffic from accidents, decision-makers should focus all their efforts on bottlenecks within our systems. The most noticeable of the jams are our highway exits. This analysis was meant to raise attention to these choke points but not solve them as that would take a more in-depth look at each exit by someone with a higher degree of domain knowledge. The region's decision-makers should employ such a person, as a civil engineer, to evaluate and suggest improvements for the worst of the areas. As far as educating the public, we see that people should keep a closer eye on pressure; when the pressure drops, you can expect the weather to worsen, and humidity levels are a high indicator of traffic than precipitation.

There is still much to be done to understand and limit traffic and accidents generally on a larger scale. If you would like to explore this dataset or topic in more depth, several other notebooks reside on Kaggle.com for your exploration. Thanks for taking the time to read this and have a good day and drive safely.
